In [ ]:
from api_key import API_KEY
from datetime import datetime as dt
from random import gauss, randint, shuffle, uniform
from time import sleep

import json
import requests

In [ ]:
DB_URL = "https://pigeonblog-db-default-rtdb.firebaseio.com"
AUTH_URL = "https://identitytoolkit.googleapis.com/v1/accounts:signUp"

LAT = {
  "min": 33.7306,
  "max": 34.1528,
  "mean": 33.9417,
  "std": 0.211176 / 2
}

LON = {
  "min": -118.3910,
  "max": -117.8382,
  "mean": -118.1146,
  "std": 0.2764025 / 2
}

### Auth

In [ ]:
post_url = f"{AUTH_URL}?key={API_KEY}"
res = requests.post(post_url, json={"returnSecureToken": True})
if res.status_code == 200:
  token = json.loads(res.text)["idToken"]
  print("ok")

### Nested schema

In [ ]:
session_id = dt.now().strftime("%Y%m%d")
pigeon_id = randint(0, 4)

sensors = { f"sen{i}": round(uniform(0.0, 16.0), 4) for i in range(8) }

measurement = {
  "timestamp": {".sv": "timestamp"},
  "lat": round(gauss(mu=LAT["mean"], sigma=LAT["std"]), 4),
  "lon": round(gauss(mu=LON["mean"], sigma=LON["std"]), 4),
  **sensors,
}

post_url = f"{DB_URL}/sessions/{session_id}/pigeons/{pigeon_id}/measurements.json?auth={token}"
res = requests.post(post_url, json=measurement)
print(res.text)

### Flat A9G

In [ ]:
datapack = {
  "timestamp": {".sv": "timestamp"},
  "pigeon": randint(0, 4),
  "lat": round(gauss(mu=LAT["mean"], sigma=LAT["std"]), 4),
  "lon": round(gauss(mu=LON["mean"], sigma=LON["std"]), 4),
  "sata": 0,
  "satb": 0,
  "temp": randint(10, 35),
  "pres": randint(750, 800),
  "alti": randint(10, 1000),
  "NH3": round(uniform(18.00, 22.0), 4),
  "OXI": round(uniform(18.00, 22.0), 4),
  "RED": round(uniform(18.00, 22.0), 4),
  "local": int(dt.now().timestamp()),
}

post_url = f"{DB_URL}/measurements.json?auth={token}"
res = requests.post(post_url, json=datapack)
print(res.text)

### ~3 minutes of test data

In [ ]:
p_list = list(range(5))
p_centers = [
  {
    "LAT": uniform(LAT["min"], LAT["max"]),
    "LON": uniform(LON["min"], LON["max"])
  } for x in p_list
]

In [ ]:
ts_date = "2023/08/20 10:00:00 +0000"
ts_date = dt.strptime(ts_date, "%Y/%m/%d %H:%M:%S %z")
ts = int(ts_date.timestamp() * 1000)

In [ ]:
for m in range(512):
  shuffle(p_list)
  for p in p_list:
    datapack = {
      "pigeon": p,
      # "timestamp": {".sv": "timestamp"},
      "timestamp": ts,
      "lat": round(gauss(mu=p_centers[p]["LAT"], sigma=LAT["std"] / 3), 4),
      "lon": round(gauss(mu=p_centers[p]["LON"], sigma=LON["std"] / 3), 4),
      "sata": 0,
      "satb": 0,
      "temp": randint(10, 35),
      "pres": randint(750, 800),
      "alti": randint(10, 1000),
      "NH3": round(uniform(18.00, 22.0), 4),
      "OXI": round(uniform(18.00, 22.0), 4),
      "RED": round(uniform(18.00, 22.0), 4),
      "local": int(dt.now().timestamp()),
    }

    post_url = f"{DB_URL}/measurements.json?auth={token}"
    res = requests.post(post_url, json=datapack)
    sleep(0.2)
  sleep(0.5)

### clean db: delete chosen range by index

In [ ]:
get_url = f'{DB_URL}/sessions.json?shallow=true'
res = requests.get(get_url)
res_obj = json.loads(res.content)
unique_days = res_obj.keys()
unique_days_array = sorted(unique_days)
[x for x in map(lambda x: dt.utcfromtimestamp(int(x)//1000).strftime("%Y-%m-%d"), unique_days_array)]

In [ ]:
first_day_to_delete_idx = 1
last_day_to_delete_idx = 1

first_day_start = int(unique_days_array[first_day_to_delete_idx])
last_day_start = int(unique_days_array[last_day_to_delete_idx])

last_day_end = last_day_start + 24 * 60 * 60 * 1000 - 1000

get_url = f'{DB_URL}/measurements.json?orderBy="timestamp"&startAt={first_day_start}&endAt={last_day_end}'
res = requests.get(get_url)
res_obj = json.loads(res.content)

print(len(res_obj))

for id in sorted(res_obj.keys()):
  del_url = f"{DB_URL}/measurements/{id}.json?auth={token}"
  res = requests.delete(del_url)